# Імпортуємо необхідні бібліотеки

In [3]:
import json
import os
import shutil
import random
import cv2

from PIL import Image

# Зчитуємо данні, що поставлялись разом з завданням

In [4]:
with open('responce.json', 'r') as json_file:
    data = json.load(json_file)

classes = {}
for image_path, angle in data.items():
    image_name = image_path.split('/')[-1]
    classes[image_name] = angle


# Створюємо основну функцію для розвороту зображень

In [5]:
def image_rotation(root, image_name, angle, new_name = True):

    input_path = root + image_name
    image = Image.open(input_path)

    rotated_image = image.rotate(angle, resample=Image.BICUBIC, expand=True)

    rotated_dir = os.path.join(root, 'rotated')
    if not os.path.exists(rotated_dir):
        os.mkdir(rotated_dir)

    if new_name:
        output_path = root + 'rotated/' + str(angle) + '_angle|' + image_name
    else:
        output_path = root + 'rotated/' + image_name

    rotated_image.save(output_path)

    rotated_image.close()

    return output_path


# Тестую функцію розвертаючи зображення поставлені з завданням

In [6]:
root = '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/'

try:
    for image_name, angle in classes.items():
        output_path = image_rotation(root, image_name, angle * -1, new_name = False)
except FileNotFoundError:
    pass

### Для забезпечення тренувальних та тестових данних застосовую data_augmentation через свою функцію. В процесі виконання функції створится по 61 екземпляру для кожного поставленого зображення. Кожен з екземплярів буде розвернутий в діапазоні від -30 до 30 градусів. Спеціально застосовуются тіж самі зображення 61 раз аби в майбутньому нейромережа ловила паттерни саме на формі(нахилу) зображення, а не за його змістом

In [14]:

root = '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/'

new_classes = {}

for angle in range(-30,31):
    try:
        for image_name, _ in classes.items():
            output_path = image_rotation(root, image_name, angle)
            new_classes[output_path] = angle
    except FileNotFoundError:
        pass


### Створений словник допоможе відсортувати зображення

In [15]:
new_classes

{'/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|0.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|1.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|2.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|3.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|4.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|5.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|6.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|7.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|8.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/rotated/rotated/-30_angle|9.png': -30,
 '/Users/aleksejkitajskij/Desk

### В якості простого способу позбавитись рамок в зображеннях, аби black box знову ж таки не вловлював паттерни з нахилу саме рамок, а не тексту, приближаємо кожне зображення. Таким чином всі навчальні зображення матимуть однакову рамку і нахиленими залишатимутся тільки текст

In [18]:
def zoom(image_path, output_path):

    image = cv2.imread(image_path)
    new_width, new_height = 1400, 1000
    center_x, center_y = image.shape[1] // 2, image.shape[0] // 2
    x1 = center_x - new_width // 2
    x2 = center_x + new_width // 2
    y1 = center_y - new_height // 2
    y2 = center_y + new_height // 2
    cropped_zoomed_image = image[y1:y2, x1:x2]
    cv2.imwrite(output_path, cropped_zoomed_image)

zoom_dic = {}

output_zoom = '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/'

for file_path, ang_folder in new_classes.items():
    img_name = file_path.split('/')[-1]
    zoom(file_path, output_zoom + img_name)

    zoom_dic[output_zoom + img_name] = ang_folder


In [19]:
zoom_dic

{'/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|0.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|1.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|2.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|3.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|4.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|5.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|6.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|7.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|8.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|9.png': -30,
 '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/zoom_img/-30_angle|10.png': -30,
 '/Users/

# Створення навчальних каталогів і сортування зображень

### Цим кодом відсортовуются зображення по різним папкам відповідно їх нахилу 

In [20]:
for file_path, ang_folder in zoom_dic.items():
    destination_folder = "/Users/aleksejkitajskij/Desktop/invoices_rotated/images/work_img/" + str(ang_folder)

    if os.path.exists(file_path):
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        shutil.copy(file_path, os.path.join(destination_folder, os.path.basename(file_path)))

    else:
        print(f"File {file_path} not exsist")



### Створюєтся тестовий та тренувальний каталог і зображення розподіляются між ними

In [21]:
root_folder = '/Users/aleksejkitajskij/Desktop/invoices_rotated/images/work_img'

test_percentage = 20

train_folder = os.path.join(root_folder, 'train')
test_folder = os.path.join(root_folder, 'test')
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):
        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png'))]
        random.shuffle(image_files)

        num_test_files = len(image_files) * test_percentage // 100

        train_files = image_files[num_test_files:]
        test_files = image_files[:num_test_files]

        for filename in train_files:
            src_path = os.path.join(subfolder_path, filename)
            dst_path = os.path.join(train_folder, subfolder, filename)
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            shutil.copy(src_path, dst_path)

        for filename in test_files:
            src_path = os.path.join(subfolder_path, filename)
            dst_path = os.path.join(test_folder, subfolder, filename)
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            shutil.copy(src_path, dst_path)
